# **Plot and Stats - wide to tidy**
---

<font size = 4>Colab Notebook for Plotting data


<font size = 4>Notebook created by [Guillaume Jacquemet](https://cellmig.org/)



# **Part 0. Before getting started**
---

Wide and tidy formats represent two principal ways of structuring tabular data:

- **Wide Format**:
  - Each row represents a subject or item.
  - Observations spread across multiple columns.
  - Suitable for data entry or presentation.
  - Example with biological repeats:
    ```
    | Subject | Cond1_Repeat1 | Cond1_Repeat2 | Cond2_Repeat1 | Cond2_Repeat2 |
    |---------|---------------|---------------|---------------|---------------|
    | 1       | ValueA        | ValueB        | ValueC        | ValueD        |
    ```

- **Tidy Format**:
  - Each column is a variable, each row an observation.
  - Suited for statistical analysis and plotting.
  - Each row represents a unique combination of variables.
  - Example with biological repeats:
    ```
    | Subject | Condition | Repeat | Value  |
    |---------|-----------|--------|--------|
    | 1       | Cond1     | 1      | ValueA |
    | 1       | Cond1     | 2      | ValueB |
    | 1       | Cond2     | 1      | ValueC |
    | 1       | Cond2     | 2      | ValueD |
    ```

Wide format is more readable for direct comparisons across a subject's measurements, while tidy format is optimized for analysis, making data transformations, summarizations, and visualizations more straightforward.




<font size = 5>**Important notes**

---

## Data Preparation and Loading Instructions

This notebook is designed to transform wide-format data into a tidy format for further analysis. To ensure smooth operation, please follow the guidelines below regarding the data format and how to use the notebook.

### File Format
- **CSV**: Data should be in CSV (Comma-Separated Values) format, easily generated from spreadsheet applications (e.g., Excel, Google Sheets) or statistical software (e.g., R, Python).
- **Copy and Paste**: Data can be directly copied and pasted from a spreedsheet software.

### Expected Data Format

Your dataset should be structured in a wide format, where:

- **Columns** represent different conditions and their repeats. The naming convention should follow the pattern `ConditionName_R#`, where `ConditionName` is a unique identifier for the condition, and `R#` indicates the repeat number (e.g., `R1`, `R2` for repeats 1 and 2, respectively).
- **Rows** represent individual subjects, samples, or experimental units with measurements under each condition and repeat.

### Example Dataset

Below is an example of how your data might look when correctly formatted. Note that missing values are allowed:

| Condition1_R1 | Condition1_R2 | Condition2_R1 | Condition2_R2 | Condition3_R1 | Condition3_R2 |
|---------------|---------------|---------------|---------------|---------------|---------------|
| 0.5           | 0.6           | 0.7           | 0.8           | 0.9           | 0.95          |
| 0.55          |               | 0.75          |               | 1.0           | 1.05          |
|               | 0.65          |               | 0.85          | 1.1           | 1.15          |
| 0.56          | 0.66          | 0.76          |               | 1.2           |               |
| 0.57          | 0.67          | 0.77          | 0.87          | 1.25          | 1.3           |

### How to Use This Notebook

1. **Load Your Data**: Use the widget provided at the beginning of the notebook to either enter the path to your CSV file or paste your data directly into the text area provided.
   - If entering a file path, ensure the path is correct and accessible by the notebook.
   - If pasting data, ensure it is tab-separated as shown in the example above.

2. **Specify the Results Folder**: Enter the path where you'd like the transformed tidy dataset and any analysis results to be saved. If the specified folder does not exist, it will be created automatically.

3. **Execute the Transformation**: Run the notebook cells in order. The notebook will automatically transform your wide-format data into a tidy format, handle missing values appropriately, and save the tidy dataset as `tidy.csv` in the specified results folder.

4. **Review and Analyze**: After transformation, you can proceed with further data analysis within the same notebook or use the tidy dataset in other analytical tools or notebooks.

### Notes

- The notebook is designed to handle datasets with varying numbers of data points across conditions and repeats by appropriately managing missing values.
- Ensure that any specific data processing or cleaning requirements are addressed either before loading the data into this notebook or within the notebook as part of the transformation process.






In [ ]:
# @title #MIT License

print("""
**MIT License**

Copyright (c) 2023 Guillaume Jacquemet

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.""")

--------------------------------------------------------
# **Part 1. Prepare the session and load your data**
--------------------------------------------------------


## **1.1. Install key dependencies**
---
<font size = 4>

In [5]:
#@markdown ##Play to load the dependancies

import ipywidgets as widgets
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import numpy as np
import itertools
from matplotlib.gridspec import GridSpec
import requests

!pip freeze > requirements.txt


# Function to calculate Cohen's d
def cohen_d(group1, group2):
    diff = group1.mean() - group2.mean()
    n1, n2 = len(group1), len(group2)
    var1 = group1.var()
    var2 = group2.var()
    pooled_var = ((n1 - 1) * var1 + (n2 - 1) * var2) / (n1 + n2 - 2)
    d = diff / np.sqrt(pooled_var)
    return d

def save_dataframe_with_progress(df, path, desc="Saving", chunk_size=50000):
    """Save a DataFrame with a progress bar."""

    # Estimating the number of chunks based on the provided chunk size
    num_chunks = int(len(df) / chunk_size) + 1

    # Create a tqdm instance for progress tracking
    with tqdm(total=len(df), unit="rows", desc=desc) as pbar:
        # Open the file for writing
        with open(path, "w") as f:
            # Write the header once at the beginning
            df.head(0).to_csv(f, index=False)

            for chunk in np.array_split(df, num_chunks):
                chunk.to_csv(f, mode="a", header=False, index=False)
                pbar.update(len(chunk))

def check_for_nans(df, df_name):
    """
    Checks the given DataFrame for NaN values and prints the count for each column containing NaNs.

    Args:
    df (pd.DataFrame): DataFrame to be checked for NaN values.
    df_name (str): The name of the DataFrame as a string, used for printing.
    """
    # Check if the DataFrame has any NaN values and print a warning if it does.
    nan_columns = df.columns[df.isna().any()].tolist()

    if nan_columns:
        for col in nan_columns:
            nan_count = df[col].isna().sum()
            print(f"Column '{col}' in {df_name} contains {nan_count} NaN values.")
    else:
        print(f"No NaN values found in {df_name}.")


import pandas as pd
import os

def save_parameters(params, file_path, param_type):
    # Convert params dictionary to a DataFrame for human readability
    new_params_df = pd.DataFrame(list(params.items()), columns=['Parameter', 'Value'])
    new_params_df['Type'] = param_type

    if os.path.exists(file_path):
        # Read existing file
        existing_params_df = pd.read_csv(file_path)

        # Merge the new parameters with the existing ones
        # Update existing parameters or append new ones
        updated_params_df = pd.merge(existing_params_df, new_params_df,
                                     on=['Type', 'Parameter'],
                                     how='outer',
                                     suffixes=('', '_new'))

        # If there's a new value, update it, otherwise keep the old value
        updated_params_df['Value'] = updated_params_df['Value_new'].combine_first(updated_params_df['Value'])

        # Drop the temporary new value column
        updated_params_df.drop(columns='Value_new', inplace=True)
    else:
        # Use new parameters DataFrame directly if file doesn't exist
        updated_params_df = new_params_df

    # Save the updated DataFrame to CSV
    updated_params_df.to_csv(file_path, index=False)


## **1.2. Mount your Google Drive**
---
<font size = 4> To use this notebook on the data present in your Google Drive, you need to mount your Google Drive to this notebook.

<font size = 4> Play the cell below to mount your Google Drive and follow the instructions.

<font size = 4> Once this is done, your data are available in the **Files** tab on the top left of notebook.

In [ ]:
#@markdown ##Play the cell to connect your Google Drive to Colab

from google.colab import drive
drive.mount('/content/gdrive')
%cd /gdrive



## **1.3. Load your dataset**
---

<font size = 4> Please ensure that your data is properly organised (see above)


In [4]:
#@markdown ##Load your dataset:

import pandas as pd
import os
from io import StringIO
import ipywidgets as widgets
from IPython.display import display, clear_output

# Initialize dataset_df as an empty DataFrame globally
dataset_df = pd.DataFrame()

# Create widgets
dataset_path_input = widgets.Text(
    value='',
    placeholder='Enter the path to your dataset',
    description='Dataset Path:',
    layout={'width': '100%'}
)

results_folder_input = widgets.Text(
    value='',
    placeholder='Enter the path to your results folder',
    description='Results Folder:',
    layout={'width': '100%'}
)

data_textarea = widgets.Textarea(
    value='',
    placeholder='Or copy and paste your tab sperated data here (direct copy and paste from a spreedsheet)',
    description='Or Paste Data:',
    layout={'width': '100%', 'height': '200px'}
)

load_button = widgets.Button(
    description='Load Data',
    button_style='success',  # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click to load the data',
)

output = widgets.Output()

# Load data function
def load_data(b):
    global dataset_df
    global Results_Folder

    with output:
        clear_output()
        Results_Folder = results_folder_input.value.strip()
        if not Results_Folder:
            Results_Folder = './Results'  # Default path if not provided
        if not os.path.exists(Results_Folder):
            os.makedirs(Results_Folder)  # Create the folder if it doesn't exist
        print(f"Results folder is located at: {Results_Folder}")

        if dataset_path_input.value.strip():
            dataset_path = dataset_path_input.value.strip()
            try:
                dataset_df = pd.read_csv(dataset_path)
                print(f"Loaded dataset from {dataset_path}")
            except Exception as e:
                print(f"Failed to load dataset from {dataset_path}: {e}")
        elif data_textarea.value.strip():
            input_data = StringIO(data_textarea.value)
            try:
                dataset_df = pd.read_csv(input_data, sep='\t')
                print("Loaded dataset from pasted tab-separated data")
            except Exception as e:
                print(f"Failed to load dataset from pasted data: {e}")
        else:
            print("No dataset path provided or data pasted. Please provide a dataset.")
            return

        # Perform a check for NaNs or any other required processing here
        check_for_nans(dataset_df, "your dataset")

        display(dataset_df.head())

# Set the button click event
load_button.on_click(load_data)

# Display the widgets
display(widgets.VBox([dataset_path_input, results_folder_input, data_textarea, load_button, output]))

In [7]:
#@markdown ##Transform your dataset to tidy format:


import pandas as pd
import numpy as np

df_wide = dataset_df

# Add an artificial index column if no natural unique identifier exists
df_wide['Index'] = range(1, len(df_wide) + 1)

# Melt the DataFrame to long format
df_long = pd.melt(df_wide, id_vars=['Index'], var_name='Condition_Repeat', value_name='Measurement')

# Drop NaN values to handle varying data points across columns
df_long = df_long.dropna(subset=['Measurement'])

# Optionally, split 'Condition_Repeat' into separate 'Condition' and 'Repeat' columns
df_long[['Condition', 'Repeat']] = df_long['Condition_Repeat'].str.rsplit('_R', n=1, expand=True)

# Drop the artificial 'Index' column if it's no longer needed, or keep it if it helps in further analysis
df_long = df_long.drop(columns=['Index'])

print(df_long)


def validate_mappings(df_long):
    condition_repeat_sets = df_long.groupby('Condition')['Repeat'].apply(set)

    # Check if all conditions have the same set of repeats
    first_set = next(iter(condition_repeat_sets), set())
    if all(repeat_set == first_set for repeat_set in condition_repeat_sets):
        print("Validation Passed: All conditions have the same set of repeats.")
    else:
        print("Warning: Not all conditions have the same set of repeats. Please check your data.")

# Call the validation function
validate_mappings(df_long)

display(df_long.head())

# Save the tidy dataset to a CSV file
df_long.to_csv(Results_Folder+'/tidy_dataset.csv', index=False)



   Condition_Repeat  Measurement   Condition Repeat
0     Condition1_R1         0.50  Condition1      1
1     Condition1_R1         0.55  Condition1      1
3     Condition1_R1         0.56  Condition1      1
4     Condition1_R1         0.57  Condition1      1
5     Condition1_R1         0.58  Condition1      1
..              ...          ...         ...    ...
91    Condition3_R2         1.95  Condition3      2
92    Condition3_R2         2.05  Condition3      2
93    Condition3_R2         2.15  Condition3      2
94    Condition3_R2         2.25  Condition3      2
95    Condition3_R2         2.35  Condition3      2

[80 rows x 4 columns]
Validation Passed: All conditions have the same set of repeats.


,Condition_Repeat,Measurement,Condition,Repeat
0,Condition1_R1,0.50,Condition1,1
1,Condition1_R1,0.55,Condition1,1
3,Condition1_R1,0.56,Condition1,1
4,Condition1_R1,0.57,Condition1,1
5,Condition1_R1,0.58,Condition1,1


# **Part 2. Version log**
---
<font size = 4>While I strive to provide accurate and helpful information, please be aware that:
  - This notebook may contain bugs.
  - Features are currently limited and will be expanded in future releases.

<font size = 4>We encourage users to report any issues or suggestions for improvement. Please check the [repository](https://github.com/guijacquemet/CellTracksColab) regularly for updates and the latest version of this notebook.


<font size = 4>**Version 0.1**
This is the first release of this notebook.

---